### This notebook is used for performing relevant data manipulations for Model Building

In [1]:
# Importing the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
import boto3
import io
s3 = boto3.client('s3')
#obj = s3.get_object(Bucket='team036vsix', Key='Dataset_final_2020_v1.csv/COVID_Traffic_2020_comb.csv')
#df = pd.read_csv(io.BytesIO(obj['Body'].read()))

In [58]:
# Reading the weather condition file into a dataframe to replace the existing weather conditions
obj = s3.get_object(Bucket='team036vsix', Key='weather_cond.csv')
df_w = pd.read_csv(io.BytesIO(obj['Body'].read()))

In [60]:
df_w.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9'], axis=1, inplace=True)


In [61]:
df_w.columns = ['Weather_Condition', 'New_name']

In [62]:
df_w.head()

,Weather_Condition,New_name
0,Rain,Rain
1,T-Storm,Storm
2,Heavy Rain / Windy,Rain
3,Light Rain with Thunder,Rain
4,Light Snow,Snow


In [103]:
df_new_both=pd.merge(df_2019, df_w, on='Weather_Condition', how='inner')

In [104]:
df_new_both.drop(['Weather_Condition'], axis=1, inplace=True)


In [105]:
df_new_both.rename(columns = {'New_name':'Weather_Condition'}, inplace = True)

In [66]:
#Checking for null values
print(pd.isnull(df_new["Weather_Condition"]))

0         False
1         False
2         False
3         False
4         False
          ...  
819702    False
819703    False
819704    False
819705    False
819706    False
Name: Weather_Condition, Length: 819707, dtype: bool


In [106]:
df_new_both.head()#1258156

,Severity,Start_Lat,Start_Lng,County,State,Temperature(F),Bump,Junction,Crossing,Stop,Traffic_Signal,Acc_date,Region,Weather_Condition
0,2,34.357560,-117.437330,San Bernardino,CA,50.0,0,0,0,0,0,2020-05-18,West,Fair
1,3,36.297780,-119.492120,Kings,CA,67.0,0,0,0,0,0,2020-05-18,West,Fair
2,2,34.359850,-117.435580,San Bernardino,CA,50.0,0,0,0,0,0,2020-05-18,West,Fair
3,2,34.207120,-118.212250,Los Angeles,CA,50.0,0,0,0,0,0,2020-05-18,West,Fair
4,2,34.207661,-118.217427,Los Angeles,CA,50.0,0,0,0,0,0,2020-05-18,West,Fair


In [107]:
# Replacing the existing temperature values/day with mean value/day
df_1_both= df_new_both.groupby(['State','County','Acc_date'])['Temperature(F)'].mean()
#df.groupby('param')['group'].nunique())
df_1_both.columns=['State','County','Date','Temp']
print(df_1_both)

State  County   Acc_date  
AL     Autauga  2019-01-08    48.4
                2019-01-17    51.8
                2019-01-23    64.4
                2019-01-29    46.4
                2019-02-06    76.3
                              ... 
WY     Uinta    2019-01-16    23.0
                2019-05-15    47.0
                2019-11-22    50.0
                2020-03-14    42.0
                2020-05-19    50.0
Name: Temperature(F), Length: 291532, dtype: float64


In [108]:
df_new1_both=pd.merge(df_new_both, df_1_both, on=['State','County','Acc_date'], how='inner')

In [109]:
df_new1_both.drop(['Temperature(F)_x'], axis=1, inplace=True)
df_new1_both.rename(columns = {'Temperature(F)_y':'Temperature'}, inplace = True)

In [110]:
df_new1_both.head(3)

,Severity,Start_Lat,Start_Lng,County,State,Bump,Junction,Crossing,Stop,Traffic_Signal,Acc_date,Region,Weather_Condition,Temperature
0,2,34.357560,-117.437330,San Bernardino,CA,0,0,0,0,0,2020-05-18,West,Fair,66.133333
1,2,34.359850,-117.435580,San Bernardino,CA,0,0,0,0,0,2020-05-18,West,Fair,66.133333
2,2,34.005587,-117.728064,San Bernardino,CA,0,0,0,0,0,2020-05-18,West,Fair,66.133333


In [111]:
#Replacing the weather condition/day with MODE value/day
df_wea_both= df_new1_both.groupby(['State','County','Acc_date'])['Weather_Condition'].agg(
    lambda x: pd.Series.mode(x)[0])

In [83]:
df_wea_both.head(10)

State  County   Acc_date  
AL     Autauga  2019-01-08    Cloudy
                2019-01-17      Rain
                2019-01-23      Rain
                2019-01-29      Rain
                2019-02-06    Cloudy
                2019-02-12      Rain
                2019-02-15      Rain
                2019-02-21    Cloudy
                2019-02-22    Cloudy
                2019-02-25    Cloudy
Name: Weather_Condition, dtype: object

In [78]:
df_new2=pd.merge(df_new1, df_wea, on=['State','County','Acc_date'], how='inner')

In [79]:
df_new2.drop(['Weather_Condition_x'], axis=1, inplace=True)
df_new2.rename(columns = {'Weather_Condition_y':'Weather_Condition'}, inplace = True)

In [84]:
df_new2.head(15)

,Severity,Start_Lat,Start_Lng,County,State,Bump,Junction,Crossing,Stop,Traffic_Signal,Acc_date,Region,Temperature,Weather_Condition
0,4,39.253570,-76.638790,Baltimore City,MD,0,0,0,0,0,2019-03-04,South,36.480000,Fair
1,4,39.253570,-76.638790,Baltimore City,MD,0,0,0,0,0,2019-03-04,South,36.480000,Fair
2,2,39.281010,-76.553391,Baltimore City,MD,0,0,0,0,0,2019-03-04,South,36.480000,Fair
3,2,39.288696,-76.533089,Baltimore City,MD,0,0,0,0,0,2019-03-04,South,36.480000,Fair
4,3,39.317059,-76.534203,Baltimore City,MD,0,0,0,0,0,2019-03-04,South,36.480000,Fair
5,2,45.345250,-118.123440,Union,OR,0,0,0,0,0,2019-10-15,West,39.125000,Fair
6,2,45.345250,-118.123440,Union,OR,0,0,0,0,0,2019-10-15,West,39.125000,Fair
7,2,45.345250,-118.123440,Union,OR,0,0,0,0,0,2019-10-15,West,39.125000,Fair
8,2,45.345250,-118.123440,Union,OR,0,0,0,0,0,2019-10-15,West,39.125000,Fair
9,2,45.346060,-118.125220,Union,OR,0,0,0,0,0,2019-10-15,West,39.125000,Fair


In [90]:
# Computing the accident count for each day
df_count=df_new2.groupby(['State','County','Acc_date','Weather_Condition','Temperature']).size().reset_index(name='Count')

In [91]:
df_count[(df_count['State']=='MD')&(df_count['County']=='Baltimore City')&(df_count['Acc_date']=='2019-03-04')]

,State,County,Acc_date,Weather_Condition,Temperature,Count
51947,MD,Baltimore City,2019-03-04,Fair,36.48,5


In [24]:
df_new3.head(3)

,State,County,Date,Weather_Condition,Temperature,Count
0,AL,Autauga,2020-01-14,Rain,66.75,0
1,AL,Autauga,2020-01-14,Rain,66.75,0
2,AL,Autauga,2020-01-14,Rain,66.75,0


In [25]:
df_gr= df_new3.groupby(['State','County','Date','Weather_Condition','Temperature','Count']).size().reset_index()

In [26]:
df_gr.rename(columns = {'Count':'Covid_Count',0:'Acc_Count'}, inplace = True)
df_gr.head()


,State,County,Date,Weather_Condition,Temperature,Covid_Count,Acc_Count
0,AL,Autauga,2020-01-02,Rain,50.000000,0,1
1,AL,Autauga,2020-01-03,Rain,63.333333,0,3
2,AL,Autauga,2020-01-04,Fair,43.000000,0,1
3,AL,Autauga,2020-01-07,Fair,53.000000,0,1
4,AL,Autauga,2020-01-08,Fair,41.000000,0,2


In [92]:
#Adding Dummy variables for the weather condition
df_dum=pd.get_dummies(df_count, prefix=['Weather_Condition'], columns=['Weather_Condition'])

In [94]:
df_dum.head()

,State,County,Acc_date,Temperature,Count,Weather_Condition_Cloudy,Weather_Condition_Fair,Weather_Condition_Fog,Weather_Condition_Rain,Weather_Condition_Snow,Weather_Condition_Storm,Weather_Condition_Windy
0,AL,Autauga,2019-01-08,48.4,1,1,0,0,0,0,0,0
1,AL,Autauga,2019-01-17,51.8,1,0,0,0,1,0,0,0
2,AL,Autauga,2019-01-23,64.4,1,0,0,0,1,0,0,0
3,AL,Autauga,2019-01-29,46.4,1,0,0,0,1,0,0,0
4,AL,Autauga,2019-02-06,76.3,1,1,0,0,0,0,0,0


In [29]:
from io import StringIO # python3; python2: BytesIO 
import boto3

bucket = 'team036vsix' # already created on S3
csv_buffer = StringIO()
df_dum.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'df_final_0328.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'DH88E8YK65DVT9R9',
  'HostId': 'xeyvPmNydQJwThbzb1LWa7J6wQNCY3nDUjaaEG1wBfjge8OlZxdQAimdO1Y1BzNnX36O72fgREw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'xeyvPmNydQJwThbzb1LWa7J6wQNCY3nDUjaaEG1wBfjge8OlZxdQAimdO1Y1BzNnX36O72fgREw=',
   'x-amz-request-id': 'DH88E8YK65DVT9R9',
   'date': 'Mon, 29 Mar 2021 02:27:52 GMT',
   'etag': '"f01f2eb75c4b3e848d0eeefbf1c8cfa3"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"f01f2eb75c4b3e848d0eeefbf1c8cfa3"'}